# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [8]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [9]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 9s 11ms/step - loss: 0.6828 - accuracy: 0.5653 - val_loss: 0.6076 - val_accuracy: 0.6845
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.5731 - accuracy: 0.7109 - val_loss: 0.5430 - val_accuracy: 0.7241
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4992 - accuracy: 0.7616 - val_loss: 0.5136 - val_accuracy: 0.7406
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4577 - accuracy: 0.7855 - val_loss: 0.4874 - val_accuracy: 0.7597
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4276 - accuracy: 0.8083 - val_loss: 0.4716 - val_accuracy: 0.7712
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4063 - accuracy: 0.8182 - val_loss: 0.4632 - val_accuracy: 0.7776
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3938 - accuracy: 0.8210 - val_loss: 0.4586 - val_accuracy: 0.7816

In [10]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 2ms/step - loss: 0.4480 - accuracy: 0.7894
Test score: 0.4480192959308624
Test accuracy: 0.789359986782074


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [11]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [12]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [14]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 17s 21ms/step - loss: 0.6457 - accuracy: 0.6183 - val_loss: 0.5070 - val_accuracy: 0.7544
Epoch 2/10
782/782 [==============================] - 15s 19ms/step - loss: 0.4602 - accuracy: 0.7815 - val_loss: 0.4489 - val_accuracy: 0.7874
Epoch 3/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3862 - accuracy: 0.8296 - val_loss: 0.4167 - val_accuracy: 0.8061
Epoch 4/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3443 - accuracy: 0.8529 - val_loss: 0.3906 - val_accuracy: 0.8239
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3122 - accuracy: 0.8682 - val_loss: 0.3837 - val_accuracy: 0.8290
Epoch 6/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2903 - accuracy: 0.8790 - val_loss: 0.3813 - val_accuracy: 0.8304
Epoch 7/10
782/782 [==============================] - 15s 20ms/step - loss: 0.2751 - accuracy: 0.8905 - val_loss: 0.3770 - val_accuracy:

In [15]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 14s 17ms/step - loss: 0.6839 - accuracy: 0.5346 - val_loss: 0.6265 - val_accuracy: 0.6993
Epoch 2/10
782/782 [==============================] - 13s 16ms/step - loss: 0.6085 - accuracy: 0.7156 - val_loss: 0.5907 - val_accuracy: 0.7396
Epoch 3/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5705 - accuracy: 0.7733 - val_loss: 0.5720 - val_accuracy: 0.7992
Epoch 4/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5484 - accuracy: 0.8007 - val_loss: 0.5545 - val_accuracy: 0.7953
Epoch 5/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5274 - accuracy: 0.8181 - val_loss: 0.5473 - val_accuracy: 0.8074
Epoch 6/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5098 - accuracy: 0.8278 - val_loss: 0.5359 - val_accuracy: 0.8097
Epoch 7/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5031 - accuracy: 0.8279 - val_loss: 0.5293 - val_accuracy:

In [19]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 12s 16ms/step - loss: 0.4557 - accuracy: 0.8437 - val_loss: 0.4937 - val_accuracy: 0.8076
Epoch 2/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4451 - accuracy: 0.8419 - val_loss: 0.4947 - val_accuracy: 0.7920
Epoch 3/10
782/782 [==============================] - 12s 16ms/step - loss: 0.4357 - accuracy: 0.8444 - val_loss: 0.4777 - val_accuracy: 0.8076
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4265 - accuracy: 0.8467 - val_loss: 0.4773 - val_accuracy: 0.8114
Epoch 5/10
782/782 [==============================] - 12s 16ms/step - loss: 0.4171 - accuracy: 0.8502 - val_loss: 0.4703 - val_accuracy: 0.8115
Epoch 6/10
782/782 [==============================] - 12s 16ms/step - loss: 0.4090 - accuracy: 0.8517 - val_loss: 0.4850 - val_accuracy: 0.8090
Epoch 7/10
782/782 [==============================] - 12s 16ms/step - loss: 0.4018 - accuracy: 0.8531 - val_loss: 0.5080 - val_accuracy: